# imports

In [1]:
from __future__ import division

In [2]:
%matplotlib notebook
import matplotlib.pyplot as plt
from schlichtanders.mymatplotlib import Centre
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn
import pandas as pd
# plt.style.use('ggplot')

In [3]:
import os, platform, sys
from pprint import pprint

In [4]:
__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

In [5]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, Session, create_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
import operator as op
from collections import defaultdict
import csv
import heapq
from copy import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
inf = float('inf')
from sklearn import cross_validation

In [7]:
import experiment_models
import experiment_util
from schlichtanders.mycontextmanagers import ignored
from schlichtanders.myobjects import Namespace, NestedNamespace
from evaluation_util import get_modes
import evaluation_util as eva

In [8]:
import theano_models as tm
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
from theano.tensor.shared_randomstreams import RandomStreams
import theano_models.probabilistic_models as pm

In [9]:
import numpy as np
import theano

In [10]:
model_names = { # sorted by optimization type
    "ml": ['baselinedet', 'baselinedetplus'],
    "annealing": ['baseline', 'baselineplus', 'mixture',
                  'planarflow', 'planarflowdet', 'radialflow', 'radialflowdet'],
    # first trials do not seem to be successfull, furthermore this needs a lot of time, maybe later on
    # "ml_exp_average": ['mixtureml', 'planarflowml', 'radialflowml'],
}
model_prefixes = reduce(op.add, model_names.values())

In [11]:
reload(experiment_util)

<module 'experiment_util' from '/home/stephan/GitProjects/theano_models/experiments/experiment1/experiment_util.pyc'>

In [12]:
Hyper = experiment_util.get_old_hyper()

# Dataset identifiers

In [13]:
datasetnames = ["boston", "concrete", "energy", "kin8nm", "powerplant", "winered", "yacht"]
datasetname = "boston"

data, error_func = experiment_util.load_and_preprocess_data(datasetname)
X, Z, VX, VZ, TX, TZ = data
example_input = X[0]
example_output = Z[0]
output_transfer = "softmax" if datasetname == "mnist" else "identity"
dataset_id = example_input, example_output, output_transfer

# Collect models and find best ones

In [14]:
reload(eva)

<module 'evaluation_util' from '/home/stephan/GitProjects/theano_models/experiments/experiment1/__file__'>

In [15]:
best_hyper = eva.get_best_hyper(["run_windows"], Hyper, model_prefixes, test_suffix=["best_val_loss", "val_error_rate"],
                               key = lambda fn, p: datasetname in fn)

AttributeError: 'Hyper' object has no attribute 'percent'

In [ ]:
best_hyper['best_val_loss']['baseline'][1][1].datasetname

In [ ]:
from experiment_util import to_pandas_dict

In [ ]:
pd.DataFrame(data=to_pandas_dict(datasetname, best_hyper)).to_csv(os.path.join(__path__, "boston_old_best_results.csv"))

save all best_hyper

# Compute Results on Test-set

In [ ]:
reload(eva)
reload(experiment_models)
reload(experiment_util)

In [ ]:
# for computing test results
pm.RNG = NestedNamespace(tm.PooledRandomStreams(pool_size=int(1e8)), RandomStreams())

In [ ]:
test_results = eva.compute_test_results(
    best_hyper, lambda hyper: experiment_util.load_and_preprocess_data(hyper.datasetname),
    model_module_id=dataset_id, n_trials=5
)

In [ ]:
test_results = eva.load_test_results(datasetname)

In [ ]:
def last_layer_to_dict(l):
    l["test_error_rate"][np.isinf(l["test_error_rate"])] = np.nan
    l['best_test_loss'][np.isinf(l['best_test_loss'])] = np.nan
    idx_nan = np.isnan(l["test_error_rate"]) | np.isnan(l['best_test_loss'])
    valid_epochs = l['best_epoch'][~ idx_nan]  # take out all unsuccessful trials
    print valid_epochs, valid_epochs.std()
    try:
        d = {"test_error_mean%i"%i: np.nanmean(l["test_error_rate"]),
             "test_nans%i"%i: idx_nan.mean(),
            "test_error_std%i"%i: np.nanstd(l["test_error_rate"]),
            "test_loss_mean%i"%i: np.nanmean(l['best_test_loss']),
            "test_loss_std%i"%i: np.nanstd(l['best_test_loss']),
            "epochs_mean%i"%i: valid_epochs.mean(),
            "epochs_std%i"%i: valid_epochs.std()}
    except Exception:
        d = {"test_error_mean%i"%i: None,
             "test_nans%i"%i: None,
            "test_error_std%i"%i: None,
            "test_loss_mean%i"%i: None,
            "test_loss_std%i"%i: None,
            "epochs_mean%i"%i: None,
            "epochs_std%i"%i: None}
    return d
        
test_results_pd = pd.DataFrame(
    data=to_pandas_dict(datasetname, test_results, last_layer_to_dict=last_layer_to_dict))
test_results_pd

# Sample the approximate posterior distribution for evaluation

In [ ]:
pd_with_index = test_results_pd.set_index(["model", "datasetname", "n_normflows"])
pd_with_index

In [ ]:
print test_results_pd.set_index(["model", "n_normflows", "datasetname"]).to_latex()

In [ ]:
reload(eva)

In [ ]:
best_hyper_samples = eva.sample_best_hyper(best_hyper, model_module_id=dataset_id)

In [ ]:
# best_hyper_samples only refers to hypers which distribution can be compared (i.e. baselinedet and such is removed)
test_attrs = sorted(best_hyper_samples.keys())
model_prefixes = sorted(best_hyper_samples[test_attrs[0]].keys())
n_normflows = sorted(best_hyper_samples[test_attrs[0]][model_prefixes[0]].keys())

lp = len(model_prefixes)
lnn = len(n_normflows)

## number of modes / histogram of modes

In [ ]:
nr_modes = eva.get_nr_modes(best_hyper_samples, threshold_d=50)
nr_modes

In [ ]:
nr_modes = eva.get_nr_modes(best_hyper_samples)
nr_modes

## Correlations between parameters

In [ ]:
reload(eva)

In [ ]:
correlations = eva.get_best_correlations(best_hyper_samples)

In [ ]:
best_hyper['best_val_loss']['planarflow'][20]

In [ ]:
def vmax(acc, a):
    return max(acc, a.max())
def vmin(acc, a):
    return min(acc, a.min())

In [ ]:
correlations0 = fmap_results(lambda x: abs(x[0]), correlations)
correlations0_vmax = -inf
correlations0_vmin = inf
correlations0_vmax = reduce_results(vmax, correlations0, correlations0_vmax)
correlations0_vmin = reduce_results(vmin, correlations0, correlations0_vmin)

In [ ]:
n_normflows

In [ ]:
model_prefixes

In [ ]:
# only correlations from best one
for test in test_attrs:
    fig, axs = plt.subplots(ncols=len(n_normflows), nrows=len(model_prefixes), sharex=True, sharey=True)
    axs_iter = axs.flat
    for name in model_prefixes:
        for nn in n_normflows:
            ax = next(axs_iter)
            ax.grid(False)
            ax.axis("off")
            try:
                im = ax.imshow(correlations0[test][name][nn],
                               vmax=correlations0_vmax, vmin=correlations0_vmin, cmap=plt.cm.hot) #, norm=Centre()
            except KeyError:  # radialflow has some problems
                pass
            
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     fig.tight_layout()
#     plt.axis('off')

## alternative plots for correlations

In [ ]:
correlations1 = fmap_results(lambda x: abs(x[1]), correlations)
correlations1_vmax = -inf
correlations1_vmin = inf
correlations1_vmax = reduce_results(vmax, correlations1, correlations1_vmax)
correlations1_vmin = reduce_results(vmin, correlations1, correlations1_vmin)

In [ ]:
# only correlations from best one
for test in test_attrs:
    fig, axs = plt.subplots(ncols=len(n_normflows), nrows=len(model_prefixes), sharex=True, sharey=True)
    axs_iter = axs.flat
    for name in model_prefixes:
        for nn in n_normflows:
            ax = next(axs_iter)
            ax.grid(False)
            ax.axis("off")
            try:
                im = ax.imshow(correlations1[test][name][nn],
                               vmax=correlations1_vmax, vmin=correlations1_vmin, cmap=plt.cm.hot) #, norm=Centre()
            except KeyError:  # radialflow has some problems
                pass
            
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     fig.tight_layout()
#     plt.axis('off')

In [ ]:
correlations_ = fmap_results(lambda x: abs(np.array(x)), correlations)
means = fmap_results(lambda x: x.mean(axis=0), correlations_)
variances = fmap_results(lambda x: x.var(axis=0), correlations_)
means_vmax = -inf
means_vmin = inf
variances_vmax = -inf
variances_vmin = 0

variances_vmax = reduce_results(vmax, variances, variances_vmax)
variances_vmin = reduce_results(vmin, variances, variances_vmin)
means_vmax = reduce_results(vmax, means, means_vmax)
means_vmin = reduce_results(vmin, means, means_vmin)

In [ ]:
# mean
for test in test_attrs:
    fig, axs = plt.subplots(ncols=len(n_normflows), nrows=len(model_prefixes), sharex=True, sharey=True)
    axs_iter = axs.flat
    for name in model_prefixes:
        for nn in n_normflows:
            ax = next(axs_iter)
            ax.grid(False)
            ax.axis("off")
            try:
                im = ax.imshow(means[test][name][nn],
                               vmax=means_vmax, vmin=means_vmin, cmap=plt.cm.hot) #, norm=Centre()
            except KeyError:  # radialflow has some problems
                pass
            
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     fig.tight_layout()
#     plt.axis('off')

In [ ]:
# variances
for test in test_attrs:
    fig, axs = plt.subplots(ncols=len(n_normflows), nrows=len(model_prefixes), sharex=True, sharey=True)
    axs_iter = axs.flat
    for name in model_prefixes:
        for nn in n_normflows:
            ax = next(axs_iter)
            ax.grid(False)
            ax.axis("off")
            try:
                im = ax.imshow(variances[test][name][nn],
                               vmax=variances_vmax, vmin=variances_vmin, cmap=plt.cm.hot)
            except KeyError:  # radialflow has some problems
                pass
            
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     fig.tight_layout()
#     plt.axis('off')